# LCA of Germany's grid status quo

In [35]:
import bw2data as bd
import bw2calc as bc

In [36]:
bd.projects.set_current("bw25_plca_grid_expansion")

In addition to the cumulated LCIA results, we want to investigate the contributions of certain grid components, materials and of the underlying directly emitting processes.

## Contributions of components and materials


Setup:

In [37]:
status_quo_node = bd.get_node(database="grid_status_quo", name="grid_status_quo")
fu = {status_quo_node: 1}

method = (
    "EF v3.1 no LT",
    "climate change no LT",
    "global warming potential (GWP100) no LT",
)

Now we calculate the LCI the standard way, but factorize the matrices because we will do additional calculations with the same matrices in the next step.

In [38]:
lca = bc.LCA(fu, method=method)
lca.lci(factorize=True)

[d3blocks] >INFO> Initialized LCA object. Demand: {761423: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x1676b0050>, <bw_processing.datapackage.Datapackage object at 0x167337510>, <bw_processing.datapackage.Datapackage object at 0x166e90210>, <bw_processing.datapackage.Datapackage object at 0x166eabf90>]
/Users/timodiepers/anaconda3/envs/plca-grid-expansion/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)
/Users/timodiepers/anaconda3/envs/plca-grid-expansion/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.51e+12)
  warnings.warn(msg, UmfpackWarning)


Calculating the LCIA results for the component and material levels:

In [39]:
component_scores = {}
for component in status_quo_node.technosphere():
    material_scores = {}
    for material in component.input.technosphere():
        lca.lcia(demand={material.input.id: material.amount * component.amount})
        material_scores[material.input["name"]] = lca.score
    component_scores[component.input["name"]] = material_scores

Saving the results in a DataFrame and cleaning up the labels:

In [40]:
import pandas as pd

def map_labels(labels, groups, other_label="other"):
    new_labels = {}
    for label in labels:
        for key, group in groups.items():
            if key in label:
                new_labels[label] = group
                break
        else:
            new_labels[label] = other_label
    return new_labels

component_groups = {
    "Overhead line": "overhead lines",
    "cable": "cables",
    "Transformer": "transformers",
    "switchgear": "switchgears",
    "Substation": "substations",
}

material_groups = {
    "aluminium": "aluminium",
    "copper": "copper",
    "iron": "iron & steel",
    "steel": "iron & steel",
    "concrete": "concrete",
    "cement": "concrete",
    "sulfur hexafluoride": "SF6",
    "polyethylene": "plastics",
    "polypropylene": "plastics",
    "plastic": "plastics",
}

df = pd.DataFrame(component_scores)

df.index = df.index.map(
    map_labels(df.index, material_groups, other_label="other materials")
)
df.columns = df.columns.map(map_labels(df.columns, component_groups))

df_components_to_materials = df.groupby(level=0).sum().T.groupby(level=0).sum().T
df_components_to_materials


,cables,overhead lines,substations,switchgears,transformers
SF6,0.000000e+00,0.000000e+00,0.000000e+00,5.096687e+08,0.000000e+00
aluminium,1.549715e+10,1.743247e+10,0.000000e+00,3.299823e+08,8.166427e+08
concrete,0.000000e+00,2.912747e+09,2.186913e+08,0.000000e+00,0.000000e+00
copper,2.977899e+09,1.508538e+09,0.000000e+00,4.855086e+07,5.357167e+08
iron & steel,4.388190e+08,1.056114e+10,8.729776e+08,4.208439e+07,1.613075e+09
other materials,4.121797e+08,6.242151e+08,0.000000e+00,5.919181e+07,7.040002e+08
plastics,5.556772e+09,9.009195e+06,0.000000e+00,0.000000e+00,0.000000e+00


## Process contributions

To get the information which processes are the direct emitters, but keep the connection to the responsible materials, we use the initially created "aggregated material" activities that essentially sum up the demands for a certain material across all grid components.


First, we collect the aggregated materials nodes:

In [41]:
aggregated_material_names = [
    "aggregated material: aluminium",
    "aggregated material: steel",
    "aggregated material: concrete",
    "aggregated material: copper",
    "aggregated material: plastics",
    "aggregated material: other",
]

aggregated_material_nodes = [
    bd.get_node(name=name) for name in aggregated_material_names
]

Now, we do the individual LCAs, and aggregate the results by the reference product of the underlying directly emitting processes:

In [42]:
import bw2analyzer as ba

lca = bc.LCA(
    {aggregated_material_nodes[0]: 1}, method=method
)  # just to build the matrices
lca.lci(factorize=True)
material_top_processes = {}
for mat in aggregated_material_nodes:
    lca.lcia(demand={mat.id: 1})
    top_processes = ba.ContributionAnalysis().annotated_top_processes(lca, limit=5000)
    score_and_product = {}
    for process in top_processes:
        if process[2]["reference product"] in score_and_product:
            score_and_product[process[2]["reference product"]] += process[0]
        else:
            score_and_product[process[2]["reference product"]] = process[0]
    material_top_processes[mat["name"]] = score_and_product

[d3blocks] >INFO> Initialized LCA object. Demand: {761424: 1}, data_objs: [<bw_processing.datapackage.Datapackage object at 0x31f538ed0>, <bw_processing.datapackage.Datapackage object at 0x31f530490>, <bw_processing.datapackage.Datapackage object at 0x177fc24d0>, <bw_processing.datapackage.Datapackage object at 0x322802d50>]
/Users/timodiepers/anaconda3/envs/plca-grid-expansion/lib/python3.11/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)
/Users/timodiepers/anaconda3/envs/plca-grid-expansion/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.51e+12)
  warnings.warn(msg, UmfpackWarning)


We save the results in a DataFrame again and clean up the names:

In [43]:
product_groups = {
    "electricity": "electricity",
    "heat": "heat",
    "transport": "transport",
    "aluminium": "aluminium (process emissions)",
    "iron": "iron & steel (process emissions)",
    "steel": "iron & steel (process emissions)",
    "coal": "coal",
    "coke": "coal",
    "clinker": "clinker",
    "diesel": "transport",
    "Gas insulated switchgear": "SF6",
    "sulfur hexafluoride": "SF6",
}

df_aggregated_materials = pd.DataFrame(material_top_processes)

df_aggregated_materials.index = df_aggregated_materials.index.map(
    map_labels(
        df_aggregated_materials.index, product_groups, other_label="other processes"
    )
)
df_aggregated_materials.columns = df_aggregated_materials.columns.map(
    map_labels(
        df_aggregated_materials.columns, material_groups, other_label="other materials"
    )
)

df_materials_to_processes = (
    df_aggregated_materials.groupby(level=0).sum().T.groupby(level=0).sum().T
)
df_materials_to_processes

,aluminium,concrete,copper,iron & steel,other materials,plastics
SF6,1.287486e+06,1.202818e+04,1.063520e+05,5.073426e+04,4.897169e+08,5.640448e+04
aluminium (process emissions),4.294089e+09,2.383359e+06,2.435253e+07,5.077888e+06,2.349886e+06,4.673412e+06
clinker,8.021734e+07,1.931035e+09,6.172646e+07,2.280361e+08,2.163575e+07,2.809964e+07
coal,2.818240e+09,8.490951e+07,2.976855e+08,2.431754e+09,6.745916e+07,3.278984e+08
electricity,1.824527e+10,3.670439e+08,2.751451e+09,1.880547e+09,5.478748e+08,1.584726e+09
heat,5.798994e+09,9.185880e+07,6.286042e+08,5.080321e+08,3.009970e+08,3.012410e+08
iron & steel (process emissions),1.293887e+08,2.345862e+07,6.520695e+07,6.623462e+09,2.416300e+07,5.971216e+07
other processes,1.606454e+09,2.106463e+08,7.647094e+08,8.938214e+08,6.680536e+08,2.932190e+09
transport,1.102312e+09,4.147565e+08,4.768622e+08,9.573120e+08,1.923402e+08,3.271841e+08


## Prepare sankey diagram

Now we have all the information we need, we just need to get the data in the right format to feed our Sankey Diagram.


In [44]:
sankey_data = []  # will be filled with tuples of (source, target, value)

Total impact:


In [45]:
df_components_to_materials.sum().sum()

63681520503.17218

Grid to components:


In [46]:
for component, score in df_components_to_materials.sum().items():
    sankey_data.append((component, "grid status quo", score))

Components to materials:


In [47]:
for target, values in df_components_to_materials.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Materials to processes:


In [48]:
for target, values in df_materials_to_processes.items():
    for source, value in values.items():
        if value != 0:
            sankey_data.append((source, target, value))

Create a DataFrame and save as .csv:

In [ ]:
df_sankey = pd.DataFrame(sankey_data, columns=["source", "target", "value"])
df_sankey.value = df_sankey.value / 1e9
df_sankey.columns = ["source", "target", "value"]
df_sankey.to_csv("data/sankey_data.csv", index=False)

The final figure for the paper was created using the Javascript library D3Blocks. The code is available on Observable: https://observablehq.com/@timo-diepers-ws/sankey-grid-status-quo-de.